In [2]:
from sdRDM import DataModel
from sdRDM_db import DBConnector, create_tables

In [3]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="ExampleDB"
)

In [4]:
# Check if there are no tables in the database
db.connection.list_tables()

[]

In [5]:
# Fetch the EnzymeML data model from the git repository
libEnzymeML = DataModel.from_git("https://github.com/EnzymeML/enzymeml-specifications.git")
libEnzymeML

Objects EnzymeMLDocument, Creator, Vessel, AbstractSpecies, Protein, Complex, Reactant,
        Reaction, ReactionElement, KineticModel, KineticParameter, Measurement, MeasurementData,
        Replicate, File
Enums   SBOTerm, DataTypes
Links   sbml

In [6]:
# Now create tables for 'Reactants' and 'Proteins'
create_tables(db_connector=db, model=libEnzymeML.Reactant)
create_tables(db_connector=db, model=libEnzymeML.Protein)

In [7]:
# Check if the tables have been created
db.connection.list_tables()

# Inspect one of the tables using Ibis
reactant_table = db.connection.table("Reactant")
reactant_table

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ name    ┃ vessel_id ┃ init_conc ┃ constant ┃ unit    ┃ uri     ┃ creator_id ┃ smiles  ┃ inchi   ┃ chebi_id ┃ ontology ┃ id     ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ !string │ !string   │ !float64  │ !string  │ !string │ !string │ !string    │ !string │ !string │ !string  │ !string  │ !int32 │
└─────────┴───────────┴───────────┴──────────┴─────────┴─────────┴────────────┴─────────┴─────────┴──────────┴──────────┴────────┘

In [8]:
# Now add an example Reactant to the database
reactant = libEnzymeML.Reactant(
    id=2,
    name="Substrate",
    vessel_id="Eppendorf Tube",
    init_conc=10.0,
    constant=True,
    unit="mM",
    creator_id="JR",
    uri="SomeURI",
    smiles="SomeSmiles",
    inchi="SomeInchi",
    chebi_id="SomeChebi",
    onotology_id="SomeOntologyID",
)

db.connection.insert("Reactant", [reactant.dict()])

In [9]:
# Check if it has been added
reactant_table

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┓
┃ name      ┃ vessel_id      ┃ init_conc ┃ constant ┃ unit    ┃ uri     ┃ creator_id ┃ smiles     ┃ inchi     ┃ chebi_id  ┃ ontology    ┃ id     ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━┩
│ !string   │ !string        │ !float64  │ !string  │ !string │ !string │ !string    │ !string    │ !string   │ !string   │ !string     │ !int32 │
├───────────┼────────────────┼───────────┼──────────┼─────────┼─────────┼────────────┼────────────┼───────────┼───────────┼─────────────┼────────┤
│ Substrate │ Eppendorf Tube │      10.0 │ true     │ mM      │ SomeURI │ JR         │ SomeSmiles │ SomeInchi │ SomeChebi │ SBO:0000247 │      2 │
└───────────┴────────────────┴───────────┴──────────┴─────────┴─────────┴────────────┴────────────┴───────────┴───────────┴─────────────┴────────┘

### Confirm using PgAdmin

<p align="center">
    <image src="./pgadmin-check.png" /> 
</p>